In [1]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from datetime import date

In [3]:
#read in data
df = pd.read_csv('./Data/timeseries.csv')
mcases = df['Cases'].max()
mdeaths = df['Deaths'].max()
mnewcases = df['New Cases'].max()

# Area Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','New Cases Last Week','New Deaths Last Week']:
    
    fig = px.area(df, x="Day0", y=col, color="Country", range_x=[20,'Day0'], range_y=[0,col])
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Line Chart

In [ ]:
for col in ['Cases', 'Deaths','Active Cases','Daily Deaths','New Cases Last Week','New Deaths Last Week']:
    
    fig = px.line(df, x="Day0", y=col, color="Country",range_x=[20,'Day0'], range_y=[0,col])
    fig.update_layout(title=col+' by Country over Time')
    fig.show()

# Animations

In [4]:
fig = px.scatter(df, x="Cases", y="Deaths", animation_frame="Day0",
           size="Daily Deaths", color="Country", hover_name="Country",animation_group="Country",text="Country",
                 range_x=[0,mcases], range_y=[0,mdeaths],trendline='ols')
fig.update_layout(title='Evolution of Deaths and Cases by Country over Time')
fig.show()

c:\users\jason collis\appdata\local\programs\python\python36\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning:

invalid value encountered in double_scalars



IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
fig = px.scatter(df, x="Cases", y="Deaths", animation_frame="Day0",
           size="Daily Deaths", color="Country", hover_name="Country",animation_group="Country",text="Country",
                 trendline='ols',log_x=True,log_y=True)
fig.update_layout(title='Evolution of Deaths and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="New Cases Last Week",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
plotly.offline.plot(fig, filename = './Saved graphs/CasesNewCasesLogStatic.html', auto_open=False)
fig.show()

In [ ]:
#paramaters
mdeaths = int(df['Deaths'].max())
mcases = int(df['Cases'].max())
mnewcases = int(df['New Cases Last Week'].max())
mnewdeaths = int(df['New Deaths Last Week'].max())

In [ ]:
fig = px.line(df, x="Cases", y="New Deaths Last Week",
              color="Country", hover_name="Country",
                 range_x=[500,mcases], range_y=[5,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()

In [ ]:
fig = px.line(df, x="Cases", y="New Cases Last Week",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[400,mnewcases],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
plotly.offline.plot(fig, filename = './Saved graphs/CasesNewCasesLogAnimated.html', auto_open=False)

fig.show()


In [ ]:
fig = px.line(df, x="Cases", y="New Deaths Last Week",animation_frame='Day0',
              color="Country", hover_name="Country",text="Country",
                 range_x=[500,mcases], range_y=[1,mnewdeaths],log_x=True,log_y=True,)
fig.update_layout(title='Evolution of New Deaths Last Week and Cases by Country over Time')
fig.show()



# Case Fatality Analysis

In [ ]:
x=np.arange(10,mcases*10,10000)
y=np.arange(10,mdeaths*10,10000)
lethality = [0.005,0.01,0.02,0.04,0.06,0.1]
greyscale = np.flip(np.cumsum([250/len(lethality) for x in lethality]))
lgs = list(zip(lethality,greyscale))
countries = ['China','USA','Germany','Italy','UK','Turkey','Spain','France']


fig = go.Figure()

for l,gs in lgs:
    fig.add_trace(go.Scatter(x=x, y=x*l, mode='lines',
        name=str(round(l*100,1))+'%',text=str(round(l*100,0))+'%',
        marker_color='rgb({},{},{})'.format(gs,gs,gs)
                            )
                 )


for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Cases'],
                             y=df[df['Country']==c]['Deaths'],
                             mode='lines+markers',text=c,name=c))

fig.update_layout(title="Deaths over Cases incl. Case Fatality Rate (CFR) Iso Lines",
                  xaxis_title="Cases",
                  yaxis_title="Deaths",
                  xaxis_type="log", yaxis_type="log",)
fig.update_xaxes(range=[1, 6])
fig.update_yaxes(range=[0.4, 4.2])  
plotly.offline.plot(fig, filename = 'CFRLines.html', auto_open=False)


fig.show()

In [ ]:
fig = px.line(df[df['Country'].isin(countries)], x="Day0", y="CFR_Total",
              color="Country", hover_name="Country",range_x=[50,85],range_y=[0,0.12])
fig.update_layout(title='Case Fatality Rate (CFR) by Country over Time')
fig.show()


In [ ]:
fig = px.line(df[df['Country'].isin(countries)], x="Day0", y="CFR_Current",
              color="Country", hover_name="Country",range_x=[50,85],range_y=[0,1])
fig.update_layout(title='Case Fatality Rate (CFR) by Country over Time')
fig.show()

In [ ]:
fig = px.scatter(df[df['Country'].isin(countries)], x="NewCases", y="NewDeaths",
                 color="Country", hover_name="Country",trendline="ols",
                 #log_x=True,log_y=True,#range_x=[200,20000],range_y=[20,2000]
                )
fig.update_layout(title='New Deaths over New Cases by Country')
fig.show()

In [ ]:
def index_X_cases(df=df,X_cases=200):
    """Produces a dataframe that sets the index to 0 at X cases and removes all preceeding observations"""
    
    # cycle through columns and reset index at X_cases
    country_dfs = []
    for c in df['Country'].unique():
        dfc = df[df['Country']==c]
        dfc = dfc[dfc['Cases']>=X_cases]
        #dfc.sort_values('Cases',inplace=True)
        dfc.reset_index(drop=True,inplace=True)
        dfc.reset_index(inplace=True)
        dfc = dfc.rename(columns={'index':'Day{}'.format(X_cases)})
        country_dfs.append(dfc)
    return pd.concat(country_dfs,axis=0)
    
# df index_X_cases 
df200 = index_X_cases(df,200)
#df200

In [ ]:
df = df200

fig = go.Figure()

for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Day200'],
                             y=df[df['Country']==c]['Cases'],
                             mode='lines+markers',text=c,name=c))

fig.update_layout(title="Cases over days reset at 200 cases by country",
                  xaxis_title="Days since 200 cases",
                  yaxis_title="Cases", 
                  yaxis_type="log",)

fig.show()

fig = go.Figure()

for l,gs in lgs:
    fig.add_trace(go.Scatter(x=x, y=x*l, mode='lines',
        name=str(round(l*100,1))+'%',text=str(round(l*100,0))+'%',
        marker_color='rgb({},{},{})'.format(gs,gs,gs)
                            )
                 )

for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Cases'],
                             y=df[df['Country']==c]['Deaths'],
                             mode='lines+markers',text=c,name=c))

fig.update_layout(title="Cases over days reset at 200 cases by country",
                  xaxis_title="Days since 200 cases",
                  yaxis_title="Cases", 
                  xaxis_type="log",
                  yaxis_type="log",)
fig.update_yaxes(range=[0.4, 4.2])  

fig.show()

In [ ]:
fig = px.line(df[df['Country'].isin(countries)], x="Cases", y="New Cases Last Week",
              color="Country", hover_name="Country",
                 range_x=[5000,mcases*1.1], range_y=[3000,140000],log_x=True,log_y=True,
             )
fig.update_layout(title='Evolution of New Cases Last Week and Cases by Country over Time')
fig.show()

In [ ]:
from scipy.optimize import curve_fit


def func(x, a, b, c):
    return a * np.exp(b * x) + c

def projection(df):
    country_dfs = []
    for c in df['Country'].unique():
        # filter for country
        dfc = df[df['Country']==c]
        
        # create vectors for curve fit
        x = dfc['Day200']
        y = dfc['Cases']
        
        # curve fit (func = exp function)
        popt, pcov = curve_fit(func, x, y, p0=[0,0,0], maxfev=5000)        
        
        # save result
        dff = pd.DataFrame({'Day200':np.arange(x.max(),50,1),'Cases_fit':func(np.arange(x.max(),50,1), *popt)})
        dff['Country']=c  
        # merge result
        dfc = dfc.merge(dff,x)
        print(dfc)
        country_dfs.append(dfc)
    
    # merge all country dfs
    dfc = pd.concat(country_dfs,axis=1)
    return dfc
projection(df200)

In [ ]:
df = projection(df200)

df#[df['Country']=='USA']

In [ ]:
fig = go.Figure()

df = projection(df200)

for c in countries:
    fig.add_trace(go.Scatter(x=df[df['Country']==c]['Day200'],
                             y=df[df['Country']==c]['Cases'],
                             mode='lines+markers',text=c,name=c,))
    #fig.add_trace(go.Scatter(x=df[df['Country']==c]['Day200'],
    #                         y=df[df['Country']==c]['Cases_fit'],
    #                         mode='lines',text=c,name=c,dash='dot'))

fig.update_layout(title="Cases and Prediction over days reset at 200 cases by country",
                  xaxis_title="Days since 200 cases",
                  yaxis_title="Cases", 
                  xaxis_type="log",
                  yaxis_type="log",)
fig.update_yaxes(range=[0.4, 4.2])  

fig.show()

In [ ]:
print('hello world')